# **# Minimax with Alpha-Beta Pruning**

In [3]:
import time
import random
from collections import Counter
# ---------------- Game helpers ---------------
WIN_LINES = [(0,1,2),(3,4,5),(6,7,8),
             (0,3,6),(1,4,7),(2,5,8),
             (0,4,8),(2,4,6)]
def check_winner(board):
    for a,b,c in WIN_LINES:
        if board[a] != " " and board[a] == board[b] == board[c]:
            return board[a]
    return None
def is_full(board):
    return all(cell != " " for cell in board)
def legal_moves(board):
    return [i for i,c in enumerate(board) if c == " "]
def next_player(board):
    counts = Counter(board)
    return "X" if counts["X"] == counts["O"] else "O"
def evaluate(board, player):
    winner = check_winner(board)
    if winner == player:
        return 1
    elif winner is not None:
        return -1
    else:
        return 0
# ---------------- Minimax ---------------
def minimax(board, player, maximizing, counter):
    counter["nodes"] += 1
    if check_winner(board) or is_full(board):
        return evaluate(board, player), None
    best_move = None
    if maximizing:
        best_score = -float("inf")
        for m in legal_moves(board):
            board[m] = player
            score, _ = minimax(board, player, False, counter)
            board[m] = " "
            if score > best_score:
                best_score, best_move = score, m
        return best_score, best_move
    else:
        opp = "O" if player == "X" else "X"
        best_score = float("inf")
        for m in legal_moves(board):
            board[m] = opp
            score, _ = minimax(board, player, True, counter)
            board[m] = " "
            if score < best_score:
                best_score, best_move = score, m
        return best_score, best_move
# ---------------- Alpha-Beta ---------------
def alphabeta(board, player, maximizing, alpha, beta, counter):
    counter["nodes"] += 1
    if check_winner(board) or is_full(board):
        return evaluate(board, player), None
    best_move = None
    if maximizing:
        value = -float("inf")
        for m in legal_moves(board):
            board[m] = player
            score, _ = alphabeta(board, player, False, alpha, beta, counter)
            board[m] = " "
            if score > value:
                value, best_move = score, m
            alpha = max(alpha, value)
            if alpha >= beta:
                break  # prune
        return value, best_move
    else:
        opp = "O" if player == "X" else "X"
        value = float("inf")
        for m in legal_moves(board):
            board[m] = opp
            score, _ = alphabeta(board, player, True, alpha, beta, counter)
            board[m] = " "
            if score < value:
                value, best_move = score, m
            beta = min(beta, value)
            if alpha >= beta:
                break  # prune
        return value, best_move
# ---------------- Run comparison ---------------
def run_comparison(board):
    player = next_player(board)
    # Minimax
    c1 = {"nodes": 0}
    t1 = time.perf_counter()
    s1, m1 = minimax(board[:], player, True, c1)
    t1 = time.perf_counter() - t1
    # Alpha-Beta
    c2 = {"nodes": 0}
    t2 = time.perf_counter()
    s2, m2 = alphabeta(board[:], player, True, -float("inf"), float("inf"), c2)
    t2 = time.perf_counter() - t2
    print("\nBoard:", "".join(board))
    print("Next to move:", player)
    print("Minimax   -> nodes:", c1["nodes"], "time:", round(t1*1000,3),"ms")
    print("AlphaBeta -> nodes:", c2["nodes"], "time:", round(t2*1000,3),"ms")
    print("Best moves:", m1, m2, " Score:", s1)
# ---------------- Demo ---------------
if __name__ == "__main__":
    # Empty board test
    board = [" "] * 9
    run_comparison(board)
    # Random mid-game boards
    random.seed(42)
    for _ in range(3):
        b = [" "] * 9
        turn = "X"
        for _ in range(random.randint(2,6)):  # play some random moves
            moves = legal_moves(b)
            if not moves or check_winner(b):
                break
            b[random.choice(moves)] = turn
            turn = "O" if turn == "X" else "X"
        run_comparison(b)


Board:          
Next to move: X
Minimax   -> nodes: 549946 time: 2212.642 ms
AlphaBeta -> nodes: 18297 time: 89.108 ms
Best moves: 0 0  Score: 0

Board: X    O   
Next to move: X
Minimax   -> nodes: 6380 time: 23.38 ms
AlphaBeta -> nodes: 365 time: 1.519 ms
Best moves: 2 2  Score: 1

Board:   OX   X 
Next to move: O
Minimax   -> nodes: 1017 time: 3.361 ms
AlphaBeta -> nodes: 117 time: 0.493 ms
Best moves: 0 0  Score: 1

Board:  O      X
Next to move: X
Minimax   -> nodes: 6380 time: 22.122 ms
AlphaBeta -> nodes: 500 time: 2.1 ms
Best moves: 2 2  Score: 1
